In [1]:
import os
import sys

# Go one level up from 'notebooks/' to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Set PYTHONPATH environment variable
os.environ["PYTHONPATH"] = project_root

# Also update sys.path so Python knows to look there for imports
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Optional: verify
print("PYTHONPATH =", os.environ["PYTHONPATH"])

PYTHONPATH = c:\Users\shich\Src\thesis\hand_emg_regression


In [2]:
from dotenv import load_dotenv
load_dotenv()

DATASET_PATH = os.environ["DATASET_PATH"]

In [3]:
from emg_hand_tracking.dataset import emg2poseSessionSlice
import emg2pose.visualization as visualization

d = emg2poseSessionSlice(f"{DATASET_PATH}/2022-04-07-1649318400-8125c-cv-emg-pose-train@2-recording-10_left.hdf5", 0, 35000)
joints = d[30][1]
visualization.get_plotly_animation_for_joint_angles([joints.numpy()])

Rendering: 1it [00:00, 27.80it/s]


In [4]:
import plotly.graph_objects as go
from emg2pose.kinematics import forward_kinematics, load_default_hand_model

# Assume joint_angles is provided above as a torch.Tensor of shape (20,)
# Convert it into a BCT tensor: Batch size = 1, Channels = 20, Time steps = 1
joint_angles_batch = joints.unsqueeze(0).unsqueeze(-1)  # shape: (1, 20, 1)

hm = load_default_hand_model()

# Compute landmarks using forward kinematics.
# Expected output shape: (1, 1, num_landmarks, 3) (typically num_landmarks=21)
landmarks_batch = forward_kinematics(joint_angles_batch, hm)

# Remove the batch and time dimensions to get (num_landmarks, 3)
landmarks = landmarks_batch.squeeze(0).squeeze(0)

# Convert to a NumPy array for Plotly plotting
landmarks_np = landmarks.detach().cpu().numpy()

fig = go.Figure()

# Plot landmarks as 3D markers
fig.add_trace(go.Scatter3d(
    x=landmarks_np[:, 0],
    y=landmarks_np[:, 1],
    z=landmarks_np[:, 2],
    mode='markers',
    marker=dict(size=4, color='blue'),
    name='Landmarks'
))

# -20

connections = [
    (5, 6), (6, 7), (7, 0), # Thumb
    (5, 8), (8, 9), (9, 10), (10, 1), # Index
    (5, 11), (11, 12), (12, 13), (13, 2), # Middle
    (5, 14), (14, 15), (15, 16), (16, 3), # Ring
    (5, 17), (17, 18), (18, 19), (19, 4), # Pinky
]

# Plot connections (lines) between the landmarks
for i, j in connections:
    fig.add_trace(go.Scatter3d(
        x=[landmarks_np[i, 0], landmarks_np[j, 0]],
        y=[landmarks_np[i, 1], landmarks_np[j, 1]],
        z=[landmarks_np[i, 2], landmarks_np[j, 2]],
        mode='lines',
        line=dict(color='black', width=2),
        showlegend=False
    ))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D Hand Pose from Joint Angles',
    margin=dict(l=0, r=0, b=0, t=30)
)

fig.show()
